KoBERT

In [ ]:
!pip install sentencepiece

In [ ]:
%pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")

In [ ]:
tokenizer('한국어 모델을 공유합니다.')

In [ ]:
import torch
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1')
text = "한국어 모델을 공유합니다."
inputs = tokenizer.batch_encode_plus([text])
out = model(input_ids = torch.tensor(inputs['input_ids']),
              attention_mask = torch.tensor(inputs['attention_mask']))
out.pooler_output.shape

In [ ]:
# 데이터셋클래스
import torch
class OurDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels
  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx].clone().detach()) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item
  def __len__(self):
    return len(self.labels)


In [ ]:
# 데이터 로드 및 분할(여기서는 성능상 일부 데이터만 사용)
import pandas as pd
from sklearn.model_selection import train_test_split
# 데이터셋 로드
url = "https://drive.google.com/uc?id=1KOKgZ4qCg49bgj1QNTwk1Vd29soeB27o"
df = pd.read_csv(url)
print(len(df))
df = df.sample(frac=0.1)
print(len(df))
X = df.review.tolist()
y = (df.rating >= 6).values.astype(int)
x_,x_test,y_,y_test = train_test_split(X,y,stratify=y,random_state=42,test_size=0.2)
x_train,x_val,y_train,y_val = train_test_split(x_,y_,stratify=y_, random_state=42,test_size=0.2)

In [ ]:
from torch.utils.data import DataLoader
# 토큰화
train_input = tokenizer(x_train, truncation=True, padding=True,return_tensors='pt')
val_input = tokenizer(x_val, truncation=True, padding=True, max_length=512,return_tensors='pt')
test_input = tokenizer(x_test, truncation=True, padding=True, max_length=512,return_tensors='pt')
# DataSet 생성
# 데이터로더 생성
# KoBERT 한국어 전용 모델 로드
# BERT를 포함한 신경망 모델
# 학습 - 미니배치
  # device
  # optimize
  # 학습 스케줄러
  # epoch수만큼 loop